# Common functions

In [233]:
from collections import namedtuple
import numpy as np
import bokeh

ParticleForce = namedtuple('ParticleForce', 'particle_id force')

class MutableParticleForce(object):
    def __init__(self, particle_id, force):
        # Particle index
        self.particle_id = particle_id
        # Particle force
        self.force = force

class ParticleForces(object):
    def __init__(self):
        self.mem_ = {}

    def get_object_ids(self):
        return self.mem_.keys()

    def get_object_forces(self, object_id):
        mem_retrieval = self.mem_.get(object_id, {})
        return [ParticleForce(idx, force) for (idx, force) in mem_retrieval.items()]

    def get_force(self, object_id, particle_id):
        return self.mem_.get(object_id, {}).get(particle_id, np.zeros(3, np.float32))

    def add_force(self, object_id, particle_id, force):
        if object_id in self.mem_:
            if particle_id in self.mem_[object_id]:
                self.mem_[object_id][particle_id] += force
            else:
                self.mem_[object_id][particle_id] = force
        else:
            self.mem_[object_id] = {particle_id : force}
            
def compute_distance_to_point(particles, point):
    if len(particles) == 0:
        return np.array([])
    particles = np.array(particles)
    point = np.array(point)
    assert len(particles.shape) == 2, ("Wrong particles shape", particles.shape)
    assert point.shape[0] == 3
    pos = particles[:, 0:3]
    distances = np.sqrt(np.sum((pos - point[np.newaxis]) ** 2, axis=-1))
    return distances
def basic_postprocess(raw_observations, particle_forces_per_step):
    '''
    Takes steps of raw observations and turns them into
    useful numpy arrays that give position and object ids.
    raw_observations should be a list of observations (can be a list of one element)
    '''
    particle_states = np.array([data['particles'] for data in raw_observations], dtype = np.float32)
    #assert len(raw_observations) == len(particle_forces_per_step) + 1, (len(raw_observations), len(particle_forces_per_step))
    ps_proto_shape = particle_states.shape
    #ps_shape = (ps_proto_shape[0], ps_proto_shape[1] / 7, 7)
    #make it num_timesteps x num_particles x 7
    #maybe already there?
    ps_shape = ps_proto_shape
    #flex_states = particle_states.reshape(ps_shape)
    flex_states = particle_states
    part_forces_list = []
    particle_ids_list = []
    for t_, raw_obs_ in enumerate(raw_observations):
        particle_ids_timestep_list = []
        forces_this_timestep = np.zeros(list(ps_shape[1:2]) + [3], dtype = np.float32)
        obj_part_idx_offset = 0
        for obj_list_idx, o in enumerate(raw_obs_['info']['observed_objects']):
            #give all the particles there the same object id
            particle_ids_timestep_list += [[obj_list_idx, int(o[1])] for _ in range(len(o[-1]))]
            if t_ < len(particle_forces_per_step):
                #add the forecs
                for part_idx in range(len(o[-1])):
                    forces_this_timestep[obj_part_idx_offset + part_idx] =\
                            particle_forces_per_step[t_].get_force(int(o[1]), part_idx)
            obj_part_idx_offset += len(o[-1])
        #add data for this timestep
        part_forces_list.append(forces_this_timestep)
        particle_ids_list.append(particle_ids_timestep_list)
    particle_forces = np.array(part_forces_list)
    particle_ids = np.array(particle_ids_list, dtype = np.float32)
    #compute_collision differences
    collision_indices = {'enter' : 14, 'stay' : 15, 'exit' : 16}
    collision_diffs = {'enter' : [], 'exit' : [], 'stay' : []}
    prev_collision_tots = None
    for raw_obs_ in raw_observations:
        these_collision_tots = {k_ : sum([o[idx_] for o in raw_obs_['info']['observed_objects']]) for k_, idx_ in collision_indices.items()}
        if prev_collision_tots is None:
            prev_collision_tots = these_collision_tots
        else:
            for k_, tots_ in these_collision_tots.items():
                collision_diffs[k_].append(tots_ - prev_collision_tots[k_])
            prev_collision_tots = these_collision_tots
    return {
        'flex_states' : flex_states,
        'particle_forces' : particle_forces,
        'particle_ids' : particle_ids,
        'collision_diffs' : collision_diffs,
    }

# This is how forces are calculated by Damian

In [254]:
'''
Some utilities to help with flex environment interface.

Eg force specification.
'''

#Damian version
def gaussian_forces(particles, force, center_particle_id, force_std, \
        force_scale = 100.0):
    # Compute and sort pairwise distances
    distances = compute_distance_to_point(particles, \
            particles[center_particle_id, 0:3])
    sorted_idx = distances.argsort()
    distances.sort()

    # Disperse forces based on distance from center particle
    object_particle_forces = []
    for distance, idx in zip(distances, sorted_idx):
        if distance > force_std:
            break
        particle_force =  1 / (np.sqrt(2.0 * np.pi) * force_std) * \
                np.exp(-distance ** 2 / (2.0 * (force_std ** 2)))
        object_particle_forces.append(MutableParticleForce(idx, particle_force))

    # Normalize forces
    force_sum = 0.0
    for particle_force in object_particle_forces:
        force_sum += particle_force.force
    for particle_force in object_particle_forces:
        if force_sum != 0.0:
            particle_force.force /= force_sum
        particle_force.force *= force_scale * force

    return object_particle_forces

def gaussian_forces_on_objects(objects, rng=None):
    if rng is None:
        rng = np.random.RandomState(seed=0)
    # CONFIG
    force_ranges = np.array([
                [-1.0, 1.0],
                [0.75, 1.5],
                [-1.0, 1.0],
                ])
    gaussian_std_min = 0.65
    gaussian_std_max = 0.9

    particle_forces = ParticleForces()

    for obj in objects:
        particles = obj[-1]
        # Pick random force and center
        force = np.array([rng.uniform(*f) for f in force_ranges])
        print(force)
        center_particle_id = rng.randint(len(particles))
        force_std = rng.uniform(gaussian_std_min, gaussian_std_max)
        force_scale = 100.0

        object_particle_forces = gaussian_forces(particles, force, \
                center_particle_id, force_std, force_scale = force_scale)

        for particle_force in object_particle_forces:
            particle_forces.add_force(int(obj[1]),
                    particle_force.particle_id,
                    particle_force.force)
    return particle_forces

def gaussian_forces_on_objects_defined(objects, force_center, force_stds, 
        force_totals, num_stds_out = 2., rng=None):
    
    if rng is None:
        rng = np.random.RandomState(seed=0)
    # CONFIG
    force_ranges = np.array([
                [-1.0, 1.0],
                [0.75, 1.5],
                [-1.0, 1.0],
                ])
    gaussian_std_min = 0.65
    gaussian_std_max = 0.9

    particle_forces = ParticleForces()

    for obj in objects:
        particles = obj[-1]
        # Pick random force and center
        force = force_totals
        center_particle_id = force_center
        force_std = force_stds
        force_scale = 100.0

        object_particle_forces = gaussian_forces(particles, force, \
                center_particle_id, force_std, force_scale = force_scale)

        for particle_force in object_particle_forces:
            particle_forces.add_force(int(obj[1]),
                    particle_force.particle_id,
                    particle_force.force)
    return particle_forces

#Damian version
def gaussian_collide_at_center(objects, rng=None):
    if rng is None:
        rng = np.random.RandomState(seed=0)
    noise_range = np.ones(3) * 0.3
    gaussian_std_min = 0.3
    gaussian_std_max = 0.6
    force_scale = 150.0

    #compute center
    center = np.zeros(3)
    for obj in objects:
        center += np.array(obj[2])
    center /= len(objects)

    # compute object particle forces
    particle_forces = ParticleForces()
    action_infos = []
    for obj in objects:
        particles = obj[-1]
        # forces point towards the center with noise
        force_direction = center - np.array(obj[2])
        force_direction += rng.uniform(-noise_range, noise_range)
        force_direction /= np.linalg.norm(force_direction)
        # choose random object particle
        particle_id = rng.randint(len(particles))
        # disperse force around particle with gaussian kernel
        gaussian_std = rng.uniform(gaussian_std_min, gaussian_std_max)
        assert gaussian_std > 0
        object_particle_forces = gaussian_forces(particles, force_direction, \
                particle_id, gaussian_std, force_scale = force_scale)

        for particle_force in object_particle_forces:
            particle_forces.add_force(int(obj[1]),
                    particle_force.particle_id,
                    particle_force.force)

        action_infos.append({
                'force_scale': force_scale,
                'force_direction': force_direction,
                'particle_id': particle_id,
                'gaussian_std': gaussian_std,
                'object': obj,
                })
    return particle_forces, action_infos

# This is how forces are calculated by Nick

In [235]:
#Nick forces
def gaussian_forces_on_space(objects, force_centers, force_stds, 
        force_totals, num_stds_out = 2., normalization_factor = 52.2):
    #TODO: it's late...normalization factor should be computed based on std dev and density
    #this should be good for std = .5
    particle_forces = ParticleForces()
    
    for _center, _std, _tot in zip(force_centers, force_stds, force_totals):
        relative_magnitudes = []
        particle_ids = []
        object_ids = []

        #for each force, iterate over each object
        for _obj in objects:
            
            #compute and sort distances to centers
            distances = compute_distance_to_point(_obj[-1], _center)
            sorted_idx = distances.argsort()
            distances.sort()

            for distance, idx in zip(distances, sorted_idx):
                if distance > _std * num_stds_out:
                    break
                particle_force = np.exp( - distance ** 2 / (2. * (_std ** 2)))
                relative_magnitudes.append(particle_force)
                particle_ids.append(idx)
                object_ids.append(int(_obj[1]))

        #now normalize and add force to object
        for _rel_mag, _part_idx, _object_idx in zip(relative_magnitudes, particle_ids, object_ids):
            force = _rel_mag * _tot / normalization_factor
            particle_forces.add_force(_object_idx, _part_idx, force)

    return particle_forces

In [279]:
with open("/Users/michael/objects.pickle", 'rb') as f:
    objects = pickle.load(f,encoding='latin1')
    
force_centers = np.array([[30,3,28],[31,1,27]])
force_stds = np.array([4, 4])
force_totals = np.array([[-20.0, -20.0, 77],
                        [54, -5.5, -0.54]])

obj_1_center = np.mean(objects[0][-1][:,0:3], axis=0)
obj_2_center = np.mean(objects[1][-1][:,0:3], axis=0)

obj_1_corner_particle_idx = np.argmax(np.linalg.norm(objects[0][-1][:, 0:3] - obj_1_center, axis=1))
obj_1_center_particle_idx = np.argmin(np.linalg.norm(objects[0][-1][:, 0:3] - obj_1_center, axis=1))

object_1_corner_particle_pos = objects[0][-1][:, 0:3][obj_1_corner_particle_idx]
object_1_center_particle_pos = objects[0][-1][:, 0:3][obj_1_center_particle_idx]

min_force_damian = 50.0
max_force_damian = 150.0
min_std_damian = 0.3
max_std_damian = 0.6

# Damian max forces
* Vary force magnitude within (50 - 150)
* Vary force std (0.3 - 0.6)

In [261]:
force_min_center = gaussian_forces_on_objects_defined(objects, obj_1_center_particle_idx, max_std_damian, np.array([min_force_damian,0,0]))
force_max_center = gaussian_forces_on_objects_defined(objects, obj_1_center_particle_idx, max_std_damian, np.array([max_force_damian,0,0]))
force_min_corner = gaussian_forces_on_objects_defined(objects, obj_1_corner_particle_idx, max_std_damian, np.array([min_force_damian,0,0]))
force_max_corner = gaussian_forces_on_objects_defined(objects, obj_1_corner_particle_idx, max_std_damian, np.array([max_force_damian,0,0]))

In [294]:
force_min_center.get_object_forces(47)

[ParticleForce(particle_id=25, force=array([86.00386581,  0.        ,  0.        ])),
 ParticleForce(particle_id=41, force=array([84.15759444,  0.        ,  0.        ])),
 ParticleForce(particle_id=29, force=array([84.1575829,  0.       ,  0.       ])),
 ParticleForce(particle_id=26, force=array([84.15757028,  0.        ,  0.        ])),
 ParticleForce(particle_id=24, force=array([84.15756026,  0.        ,  0.        ])),
 ParticleForce(particle_id=21, force=array([84.15755808,  0.        ,  0.        ])),
 ParticleForce(particle_id=9, force=array([84.15754894,  0.        ,  0.        ])),
 ParticleForce(particle_id=42, force=array([82.35095216,  0.        ,  0.        ])),
 ParticleForce(particle_id=22, force=array([82.35094072,  0.        ,  0.        ])),
 ParticleForce(particle_id=45, force=array([82.35091601,  0.        ,  0.        ])),
 ParticleForce(particle_id=30, force=array([82.35090938,  0.        ,  0.        ])),
 ParticleForce(particle_id=13, force=array([82.35090697,  

In [295]:
forces_nick = gaussian_forces_on_space(objects, [object_1_corner_particle_pos, [0,0,0]], np.array([2,2]), 
                                       np.array([[max_force_damian,0,0],[max_force_damian,0,0]]), num_stds_out = 2., normalization_factor = 52.2)

# Run Nick calculation

In [296]:
forces_nick.get_object_forces(47)

[ParticleForce(particle_id=51, force=array([0.86136841, 0.        , 0.        ])),
 ParticleForce(particle_id=50, force=array([0.85707856, 0.        , 0.        ])),
 ParticleForce(particle_id=49, force=array([0.84948612, 0.        , 0.        ])),
 ParticleForce(particle_id=48, force=array([0.83867759, 0.        , 0.        ])),
 ParticleForce(particle_id=35, force=array([0.83250802, 0.        , 0.        ])),
 ParticleForce(particle_id=34, force=array([0.82836182, 0.        , 0.        ])),
 ParticleForce(particle_id=33, force=array([0.82102387, 0.        , 0.        ])),
 ParticleForce(particle_id=32, force=array([0.81057747, 0.        , 0.        ])),
 ParticleForce(particle_id=19, force=array([0.80147777, 0.        , 0.        ])),
 ParticleForce(particle_id=18, force=array([0.79748617, 0.        , 0.        ])),
 ParticleForce(particle_id=17, force=array([0.79042161, 0.        , 0.        ])),
 ParticleForce(particle_id=55, force=array([0.78452285, 0.        , 0.        ])),
 Par